In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
%matplotlib inline

# Mounting and Installation of Package: Only on Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My\ Drive/Colab\ Notebooks/adversarial_reisz

In [ ]:
!python setup.py develop

# Library Imports

In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import scipy
import scipy.special

# Data Generation

Data are generated as:
\begin{align}
x[1:] \sim~& U[-1, 1]^{d-1}\\
x[0] \sim~& Binomial(1, .5 + .3 \cdot g(x))\\
y \sim~& x[0] + g(x) + N(0, 1)
\end{align}
We consider several non-linear functions $g$. We then want to learn some average moment $E[m(x, y; f)]$ of the regression function 
\begin{equation}
f(x) = E[y\mid x] = x[0] + g(x).
\end{equation}

In [ ]:
def nonlin(x):
    # return 1.5 * (x[:, 1]**2)
    # return x[:, 1]
    # return scipy.special.expit(10 * x[:, 1])
    return np.abs(x[:, 1])
    # return 1.5*scipy.special.expit(10 * x[:, 1]) - 1.5*scipy.special.expit(10 * x[:, 2])

def true_propensity(X):
    return .5 + .3 * nonlin(X)

def true_f(X):
    return X[:, 0] + nonlin(X)

def gen_data(n, p):
    X = np.random.uniform(-1, 1, size=(n, p))
    X[:, 0] = np.random.binomial(1, true_propensity(X))
    y = true_f(X) + np.random.normal(size=(n,))
    return X, y

np.random.seed(123)
n = 2000
p = 3
X, y = gen_data(n, p)
X_test, y_test = gen_data(n, p)

# Moment Definition

### ATE Moment

In [ ]:
def moment_fn(x, test_fn):
    t1 = test_fn(np.hstack([np.ones((x.shape[0], 1)), x[:, 1:]]))
    t0 = test_fn(np.hstack([np.zeros((x.shape[0], 1)), x[:, 1:]]))
    return t1 - t0

def true_reisz(x, propensity):
    return (x[:, 0]==1) / propensity(x) - (x[:, 0]==0)/(1 - propensity(x))

### Policy Moment

In [ ]:
policy = lambda x: scipy.special.expit(10*x[:, 1])

def moment_fn(x, test_fn):
    t1 = test_fn(np.hstack([np.ones((x.shape[0], 1)), x[:, 1:]]))
    t0 = test_fn(np.hstack([np.zeros((x.shape[0], 1)), x[:, 1:]]))
    p1 = policy(x)
    if len(t1.shape) > 1:
        p1 = p1.reshape(-1, 1)
    return t1 * p1 + t0 * (1 - p1) 

def true_reisz(x, propensity):
    p1 = policy(x)
    prop1 = propensity(x)
    return p1 * (x[:, 0]==1) / prop1 + (1 - p1) * (x[:, 0]==0)/(1 - prop1)

### X-Transportation

The true Reisz is more complex here due to the correlation of T=X[:, 0] and X[:, 1]. We have, let $t=X[:, 0]$ and $x=X[:, 1]$. Then the relevant ratios are:
\begin{align}
\frac{f_{trans}(t, x)}{f(t, x)} = \frac{Pr[T=t|X=t^{-1}(x)] f(t^{-1}(x)) (t^{-1}(x))'}{Pr[T=t|X=x] f(x)} 
\end{align}
When $x$ is drawn from the uniform distribution, then the density if constant and the above simplifies to:
\begin{align}
\frac{f_{trans}(t, x)}{f(t, x)}= \frac{Pr[T=t|X=t^{-1}(x)] (t^{-1}(x))'}{Pr[T=t|X=x]}
\end{align}

In [ ]:
trans = lambda x: (1 + x)**(1/2)/2**(1/2)
invtrans = lambda u: 2 * (u**2) - 1
grad_invtrans = lambda u: 4 * u

def moment_fn(x, test_fn):
    tx =  np.hstack([x[:, [0]], trans(x[:, [1]]), x[:, 2:]])
    return test_fn(tx) - test_fn(x)

def true_reisz(x, propensity=None):
    invtx = np.hstack([x[:, [0]], invtrans(x[:, [1]]), x[:, 2:]])
    out0 = np.clip(propensity(invtx) * grad_invtrans(x[:, 1]) / propensity(x), 0, np.inf)
    out1 = np.clip((1 - propensity(invtx)) * grad_invtrans(x[:, 1]) / (1 - propensity(x)), 0, np.inf)
    return out0 * x[:, 0] + out1 * (1 - x[:, 0]) - 1

# Adversarial Reisz Estimator

In [ ]:
from advreisz.linear import SparseLinearAdvRiesz
from advreisz.linear.utilities import SeparateFeaturizer, CoordinatePolynomialFeatures

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.kernel_approximation import Nystroem, RBFSampler
from sklearn.pipeline import Pipeline

# feat = Pipeline([('p', CoordinatePolynomialFeatures(degree=3))])
feat = Pipeline([('p', PolynomialFeatures(degree=4, include_bias=False))])
# feat = Pipeline([('nys', Nystroem(n_components=100, kernel='rbf', gamma=5, degree=3))])
# feat = Pipeline([('nys', RBFSampler(n_components=100, gamma=2))])
feat.steps.append(('s', StandardScaler()))
feat.steps.append(('cnt', PolynomialFeatures(degree=1, include_bias=True)))
# feat = Pipeline([('p', SeparateFeaturizer(feat))])
# feat.steps.append(('cnt', PolynomialFeatures(degree=1, include_bias=True)))
splin = SparseLinearAdvRiesz(moment_fn, featurizer=feat,
                             n_iter=50000, lambda_theta=0.01, B=10,
                             tol=0.00001).fit(X)

In [ ]:
coefs = splin.coef
print("Maximum violation: ", splin.max_violation_)
#print("Estimated non-zero coefs: ", np.array(list(zip(splin.get_feature_names(), coefs))))
print("Min/Max: ", splin.min_response_loss_, splin.max_response_loss_)
print("Dualit Gap: ", splin.duality_gap_)
print("Iters: ", splin.n_iters_)

In [ ]:
plt.plot(np.log(np.array(splin.duality_gaps)))
plt.show()

In [ ]:
plt.plot(np.exp(np.array(splin.log_theta_list)))
plt.show()

In [ ]:
plt.plot(np.exp(np.array(splin.log_w_list)))
plt.show()

# Evaluation of Learned Reisz Representer

In [ ]:
plt.figure()
treated = (X_test[:, 0] == 1)
plt.scatter(true_reisz(X_test[treated], true_propensity), splin.predict(X_test[treated]))
plt.xlabel('true reisz')
plt.ylabel('estimated reisz')
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
error = true_reisz(X_test, true_propensity) - splin.predict(X_test)
plt.title("RMSE: {:.3f}, MAE: {:.3f}".format(np.sqrt(np.mean(error**2)), np.mean(np.abs(error))))
for t in [0, 1]:
    plt.subplot(1, 2, t + 1)
    treated = (X_test[:, 0]==t)
    Xtreated = X_test[treated].copy()
    Xtreated[:, 1] = np.linspace(-1, 1, Xtreated.shape[0])
    plt.plot(Xtreated[:, 1], splin.predict(Xtreated), label='est a({}, X)'.format(t))
    plt.plot(Xtreated[:, 1], true_reisz(Xtreated, true_propensity), label='true a({}, X)'.format(t))
    plt.legend()
plt.show()

# Debiased Average Moment Estimate

Applying the learned Reisz representer in order to debias a preliminary regression based model of the ATE

In [ ]:
def mean_ci(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
from sklearn.linear_model import LassoCV, Lasso, LogisticRegressionCV
from sklearn.pipeline import Pipeline

est = Pipeline([('p', PolynomialFeatures(degree=1)), ('l', Lasso(alpha=.1))]).fit(X, y)
propensity = Pipeline([('p', PolynomialFeatures(degree=1)), ('l', LogisticRegressionCV())]).fit(X[:, 1:], X[:, 0])

In [ ]:
a_test = splin.predict(X_test)
invp_test = true_reisz(X_test, lambda x: propensity.predict_proba(x[:, 1:])[:, 1])
true_ate = np.mean(moment_fn(X_test, true_f))
naive_ate = np.mean(y_test[X_test[:, 0]==1]) - np.mean(y_test[X_test[:, 0]==0])
biased_ate = np.mean(moment_fn(X_test, est.predict))
ips_ate = np.mean(invp_test * y_test)
reisz_ate = np.mean(a_test * y_test)
dr_ate = biased_ate + np.mean(invp_test * (y_test - est.predict(X_test)))
dr_reisz_ate, dr_reisz_low, dr_reisz_up = mean_ci(moment_fn(X_test, est.predict) + a_test * (y_test - est.predict(X_test)))

In [ ]:
print("True ATE: {:.3f}".format(true_ate))
print("Mean of Treated - Mean of Untreated: {:.3f}".format(naive_ate))
print("Lasso Regression based Estimate: {:.3f}".format(biased_ate))
print("IPS estimate with explicit propensity based reisz estimate (i.e. mean(a(X) Y)): {:.3f}".format(ips_ate))
print("IPS estimate with adversarial reisz: {:.3f}".format(reisz_ate))
print("DR estimate with explict propensity based reisz estimate: {:.3f}".format(dr_ate))
print("DR estimate with adversarial reisz: {:.3f} ({:.3f}, {:.3f})".format(dr_reisz_ate, dr_reisz_low, dr_reisz_up))

In [ ]:
dr_reisz_ate = []
reisz_ate = []
biased_ate = []
dr_reisz_cov = []
reisz_cov = []
for exp in range(1000):
    Xboot, yboot = gen_data(n, p)
    aboot = splin.predict(Xboot)
    drm, drl, dru = mean_ci(moment_fn(Xboot, est.predict) + aboot * (yboot - est.predict(Xboot)))
    rm, rl, ru = mean_ci(aboot * yboot)
    dr_reisz_ate.append(drm)
    reisz_ate.append(rm)
    dr_reisz_cov.append((true_ate <= dru) & (true_ate >= drl))
    reisz_cov.append((true_ate <= ru) & (true_ate >= rl))

In [ ]:
plt.title("Mean advDR: {:.3f}, Coverage advDR: {:.3f}, "
          "Mean advIPS: {:.3f}, Coverage advIPS: {:.3f}".format(np.mean(dr_reisz_ate),
                                                               np.mean(dr_reisz_cov),
                                                               np.mean(reisz_ate),
                                                               np.mean(reisz_cov)))
plt.hist(np.array(dr_reisz_ate), alpha=.5, label='advDR')
plt.hist(np.array(reisz_ate), alpha=.5, label='advIPS')
plt.legend()
plt.show()